In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import os, json
import cv2

import torch
from torchvision import models
import torch.nn as nn
#transforms
#from torchvision.transforms import Compose, Normalize, ToTensor
from torch.autograd import Variable
import torch.nn.functional as F
#from facenet_pytorch import InceptionResnetV1


import argparse


from sklearn.cluster import KMeans

from pytorch_grad_cam import (
    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
    AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
    LayerCAM, FullGrad, GradCAMElementWise
)
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import (
    show_cam_on_image, deprocess_image, preprocess_image
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from flashtorch.utils import apply_transforms, load_image
from torchvision import transforms


In [ ]:
# resize and take the center part of image to what our model expects
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transf(img).unsqueeze(0)

In [ ]:
def apply_threshold(cam, threshold):
    return np.where(cam >= threshold, cam, 0)

In [ ]:
def ensemble_cam(image_path):
    rgb_img_original = cv2.imread(image_path, 1)[:, :, ::-1]

    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    face_model = "models/facePADceleb_adamW_new.pt"
    model = torch.load(face_model)
    model.to(device)
    
    rgb_img_pil = Image.fromarray(rgb_img_original)
    img_t = get_input_tensors(rgb_img_pil)
    model.eval()

    logits = model(img_t.cuda())
    
    probs = F.softmax(logits, dim=1)
    probs2 = probs.topk(2)
    
    target_class = probs2.indices[0, 0].item() 

    target_layers = [model.features.denseblock4.denselayer24.conv2]
    targets = [ClassifierOutputTarget(target_class)]

    cam1 = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam1 = cam1(input_tensor=img_t.cuda(), targets=targets)
    grayscale_cam1 = grayscale_cam1[0, :]
    
    max_val1 = grayscale_cam1.max()
    if max_val1 > 0:  # Avoid division by zero
       grayscale_cam1_norm = grayscale_cam1 / max_val1
    else:
       grayscale_cam1_norm = grayscale_cam1  # Keep it unchanged if max is zero
    #grayscale_cam1_norm = grayscale_cam1 / grayscale_cam1.max()
    
    cam2 = HiResCAM(model=model, target_layers=target_layers)
    grayscale_cam2 = cam2(input_tensor=img_t.cuda(), targets=targets)
    grayscale_cam2 = grayscale_cam2[0, :]
    
    max_val2 = grayscale_cam2.max()
    if max_val2 > 0:  # Avoid division by zero
       grayscale_cam2_norm = grayscale_cam2 / max_val2
    else:
       grayscale_cam2_norm = grayscale_cam2  # Keep it unchanged if max is zero
    #grayscale_cam2_norm = grayscale_cam2 / grayscale_cam2.max()
    
    cam3 = GradCAMPlusPlus(model=model, target_layers=target_layers)
    grayscale_cam3 = cam3(input_tensor=img_t.cuda(), targets=targets)
    grayscale_cam3 = grayscale_cam3[0, :]
    
    max_val3 = grayscale_cam3.max()
    if max_val3 > 0:  # Avoid division by zero
       grayscale_cam3_norm = grayscale_cam3 / max_val3
    else:
       grayscale_cam3_norm = grayscale_cam3  # Keep it unchanged if max is zero
    #grayscale_cam3_norm = grayscale_cam3 / grayscale_cam3.max()

    grayscale_cam = (grayscale_cam1_norm + grayscale_cam2_norm + grayscale_cam3_norm) / 3
    

  
    threshold_1 = np.percentile(grayscale_cam, 90)

    grayscale_cam_avg = apply_threshold(grayscale_cam, threshold_1)
    
    return rgb_img_original, device, model, target_class, img_t, grayscale_cam_avg


In [ ]:


def apply_mask(image, heatmap, mask_type='black'):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
     
    threshold = np.percentile(heatmap, 0)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))  # Resize heatmap to image size
    mask = heatmap > threshold  # Create a binary mask of important regions
    
    
    
    # Apply masking
    perturbed_image = image.copy()
    if mask_type == 'black':
        perturbed_image[mask] = 0  # Set important regions to black
    elif mask_type == 'mean':
        mean_pixel_value = np.mean(image, axis=(0, 1), keepdims=True)
        perturbed_image[mask] = mean_pixel_value  # Set important regions to mean pixel value
    elif mask_type == 'blur':
        blurred = cv2.GaussianBlur(image, (21, 21), 0)
        perturbed_image[mask] = blurred[mask]  # Apply blur to important regions
    
    pil_image = Image.fromarray(perturbed_image.astype('uint8'))
    return pil_image


In [ ]:
def shift_mask(mask, shift_y, shift_x):
    """
    Shifts the mask by given offsets. Wraps around if necessary.
    
    Args:
        mask: Input binary mask (2D array).
        shift_y: Vertical shift (positive for down, negative for up).
        shift_x: Horizontal shift (positive for right, negative for left).
    
    Returns:
        Shifted mask of the same size.
    """
    # Perform shift with wrapping using np.roll
    shifted_mask = np.roll(mask, shift_y, axis=0)  # Shift vertically
    shifted_mask = np.roll(shifted_mask, shift_x, axis=1)  # Shift horizontally
    return shifted_mask

def apply_mask_random(image, heatmap, mask_type='black'):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
     
    threshold = np.percentile(heatmap, 0)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))  # Resize heatmap to image size
    mask = heatmap > threshold  # Create a binary mask of important regions
    
    # Generate random shifts
    np.random.seed(42)
    #shift_y = np.random.randint(-mask.shape[0] // 2, mask.shape[0] // 2)
    #shift_x = np.random.randint(-mask.shape[1] // 2, mask.shape[1] // 2)
    
    shift_y = np.random.randint(-image.shape[0], image.shape[0])
    shift_x = np.random.randint(-image.shape[1], image.shape[1])
    
    #print("Vertical:", shift_y)
    #print("Horizontal:", shift_x)

    # Shift the mask
    shifted_mask = shift_mask(mask, shift_y, shift_x)
    
    # Apply masking
    perturbed_image = image.copy()
    if mask_type == 'black':
        perturbed_image[shifted_mask] = 0  # Set important regions to black
    elif mask_type == 'mean':
        mean_pixel_value = np.mean(image, axis=(0, 1), keepdims=True)
        perturbed_image[mask] = mean_pixel_value  # Set important regions to mean pixel value
    elif mask_type == 'blur':
        blurred = cv2.GaussianBlur(image, (21, 21), 0)
        perturbed_image[mask] = blurred[mask]  # Apply blur to important regions
    
    pil_image = Image.fromarray(perturbed_image.astype('uint8'))
    return pil_image


In [ ]:
def evaluate_confidence_drop(model, device, original_image, perturbed_image, target_class):
    """
    Measure the drop in confidence between the original and perturbed image.
    Arguments:
    - model: Trained model
    - original_image: Original input image (tensor)
    - perturbed_image: Image with masked important regions (tensor)
    - target_class: Target class for the prediction
    """
    model.to(device)
    model.eval()

    # Original confidence
    original_output = model(original_image.cuda())
    original_confidence = F.softmax(original_output, dim=1)[0, target_class].item()

    # Perturbed confidence
    perturbed_output = model(perturbed_image.cuda())
    perturbed_confidence = F.softmax(perturbed_output, dim=1)[0, target_class].item()

    # Calculate confidence drop
    confidence_drop = original_confidence - perturbed_confidence
    return confidence_drop

In [ ]:
def ensemble_evaluate(image_path):
    
 
  img, device, model, target_class, img_t, cam = ensemble_cam(image_path)
 
    
  perturbed_image = apply_mask(img, cam)

  perturbed_image_t = get_input_tensors(perturbed_image)
    
  random_image = apply_mask_random(img, cam)

  random_image_t = get_input_tensors(random_image)

    
  
   
  ensemble_confidence_drop = evaluate_confidence_drop(model, device, img_t, perturbed_image_t, target_class)
   
  random_confidence_drop = evaluate_confidence_drop(model, device, img_t, random_image_t, target_class)
  

  return ensemble_confidence_drop, random_confidence_drop

 

In [ ]:
# Define the folder containing subfolders of images
root_folder = "Celeba_spoof/test"

In [ ]:



def process_folder_one(folder_path):
    """
    Process all images in a folder and its subfolders, calculate average confidence drops.
    """
    ensemble_confidence_drop_avg = []
    random_confidence_drop_avg = []
  
    

    for subdir, _, files in os.walk(folder_path):
       for file in files:  # Loop through each file in the current directory
            image_path = os.path.join(subdir, file)
            drop_case1, drop_case2 = ensemble_evaluate(image_path)
            ensemble_confidence_drop_avg.append(drop_case1)
            random_confidence_drop_avg.append(drop_case2)
          
            
            
            
            
    avg_drop_ensemble = np.mean(ensemble_confidence_drop_avg)
    avg_drop_random = np.mean(random_confidence_drop_avg)

    return avg_drop_ensemble, avg_drop_random

   
        
        
# Process the folder and calculate average confidence drops
avg_case1, avg_case2 = process_folder_one(root_folder)

print(f"Average Confidence Drop (Ensemble): {avg_case1}")
print(f"Average Confidence Drop (random): {avg_case2}")

In [ ]:
def only_mask(image, heatmap):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
    threshold = np.percentile(heatmap, 0)
    # Convert to NumPy array
    #image = np.array(image)

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

    # Create a binary mask
    mask = heatmap > threshold

    # Ensure the image is a proper copy
    perturbed_image = np.copy(image)

    # Initialize the result image with black
    result = np.zeros_like(perturbed_image)

  

    # Copy the mask region to the result
    result[mask] = perturbed_image[mask]
    
    
    pil_image = Image.fromarray(result.astype('uint8'))
    return pil_image


In [ ]:
def only_mask_random(image, heatmap):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
     
    threshold = np.percentile(heatmap, 0)
    # Convert the PIL image to a numpy array
    #image = np.array(image)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))  # Resize heatmap to image size
    mask = heatmap > threshold  # Create a binary mask of important regions
    
    # Generate random shifts
    np.random.seed(42)
    #shift_y = np.random.randint(-mask.shape[0] // 2, mask.shape[0] // 2)
    #shift_x = np.random.randint(-mask.shape[1] // 2, mask.shape[1] // 2)
    
    shift_y = np.random.randint(-image.shape[0], image.shape[0])
    shift_x = np.random.randint(-image.shape[1], image.shape[1])
    

    # Shift the mask
    shifted_mask = shift_mask(mask, shift_y, shift_x)
    
     # Ensure the image is a proper copy
    perturbed_image = np.copy(image)

    # Initialize the result image with black
    result = np.zeros_like(perturbed_image)

  

    # Copy the mask region to the result
    result[shifted_mask] = perturbed_image[shifted_mask]
    
    
    pil_image = Image.fromarray(result.astype('uint8'))
    return pil_image



In [ ]:
def ensemble_mask_evaluate(image_path):
    
 
  img, device, model, target_class, img_t, cam = ensemble_cam(image_path)
 
    
  perturbed_image = only_mask(img, cam)

  perturbed_image_t = get_input_tensors(perturbed_image)
    
  random_image = only_mask_random(img, cam)

  random_image_t = get_input_tensors(random_image)

    
  
   
  ensemble_confidence_drop = evaluate_confidence_drop(model, device, img_t, perturbed_image_t, target_class)
   
  random_confidence_drop = evaluate_confidence_drop(model, device, img_t, random_image_t, target_class)
  

  return ensemble_confidence_drop, random_confidence_drop

 

In [ ]:
def process_folder_mask(folder_path):
    """
    Process all images in a folder and its subfolders, calculate average confidence drops.
    """
    ensemble_confidence_drop_avg = []
    random_confidence_drop_avg = []
  
    

    for subdir, _, files in os.walk(folder_path):
       for file in files:  # Loop through each file in the current directory
            image_path = os.path.join(subdir, file)
            drop_case1, drop_case2 = ensemble_mask_evaluate(image_path)
            ensemble_confidence_drop_avg.append(drop_case1)
            random_confidence_drop_avg.append(drop_case2)
          
            
            
            
            
    avg_drop_ensemble = np.mean(ensemble_confidence_drop_avg)
    avg_drop_random = np.mean(random_confidence_drop_avg)

    return avg_drop_ensemble, avg_drop_random

   
        
        
# Process the folder and calculate average confidence drops
avg_case1, avg_case2 = process_folder_mask(root_folder)

print(f"Average Confidence Drop (Ensemble): {avg_case1}")
print(f"Average Confidence Drop (random): {avg_case2}")

In [ ]:
def evaluate(model, device, original_image, perturbed_image, random_perturbed_image):

  model.to(device)
  model.eval()

  # Original confidence
  original_output = model(original_image.cuda())
  original_probs = F.softmax(original_output, dim=1)
  original_top_prob, original_top_class = torch.max(original_probs, dim=1)
  original_top_prob = original_top_prob.item()
  original_top_class = original_top_class.item()



  # Perturbed confidence
  perturbed_output = model(perturbed_image.cuda())
  perturbed_probs = F.softmax(perturbed_output, dim=1)
  perturbed_top_prob, perturbed_top_class = torch.max(perturbed_probs, dim=1)
  perturbed_top_prob = perturbed_top_prob.item()
  perturbed_top_class = perturbed_top_class.item()
  
  perturbed_confidence = F.softmax(perturbed_output, dim=1)[0, original_top_class].item()

  
  # random Perturbed confidence
  random_perturbed_output = model(random_perturbed_image.cuda())
  random_perturbed_probs = F.softmax(random_perturbed_output, dim=1)
  random_perturbed_top_prob, random_perturbed_top_class = torch.max(random_perturbed_probs, dim=1)
  random_perturbed_top_prob = random_perturbed_top_prob.item()
  random_perturbed_top_class = random_perturbed_top_class.item()
  
  random_perturbed_confidence = F.softmax(random_perturbed_output, dim=1)[0, original_top_class].item()

  return original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence

In [ ]:
def ensemble_data(image_path):
  
  img, device, model, target_class, img_t, cam = ensemble_cam(image_path)
 
    
  perturbed_image = apply_mask(img, cam)

  perturbed_image_t = get_input_tensors(perturbed_image)
    
  random_image = apply_mask_random(img, cam)

  random_image_t = get_input_tensors(random_image)

    
  #target_class = probs5.indices[0, 0].item() 
   
  #SDD_confidence_drop = evaluate_confidence_drop(model, img_t, perturbed_image_t, target_class)
   
  #random_confidence_drop = evaluate_confidence_drop(model, img_t, random_image_t, target_class)
  original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence = evaluate(model, device, img_t, perturbed_image_t, random_image_t) 
  
  return  original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence



In [ ]:
def ensemble_mask_data(image_path):
  
  img, device, model, target_class, img_t, cam = ensemble_cam(image_path)
 
    
  perturbed_image = only_mask(img, cam)

  perturbed_image_t = get_input_tensors(perturbed_image)
    
  random_image = only_mask_random(img, cam)

  random_image_t = get_input_tensors(random_image)

    
  #target_class = probs5.indices[0, 0].item() 
   
  #SDD_confidence_drop = evaluate_confidence_drop(model, img_t, perturbed_image_t, target_class)
   
  #random_confidence_drop = evaluate_confidence_drop(model, img_t, random_image_t, target_class)
  original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence = evaluate(model, device, img_t, perturbed_image_t, random_image_t) 
  
  return  original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence



In [ ]:
import pandas as pd



def process_folder_two(folder_path):
    """
    Process all images in a folder and its subfolders, calculate average confidence drops.
    """
    #sdd_confidence_drop_avg = []
    #random_confidence_drop_avg = []
    image_name = []
    original_top_class_all = []
    original_top_prob_all = []
    perturbed_top_class_all = []
    perturbed_top_prob_all = []
    perturbed_confidence_all = []
    random_perturbed_top_class_all = []
    random_perturbed_top_prob_all = []
    random_perturbed_confidence_all = []

    for subdir, _, files in os.walk(folder_path):
       for file in files:  # Loop through each file in the current directory
            image_name.append(file)
            image_path = os.path.join(subdir, file)
            #drop_case1, drop_case2 = sdd_generate(image_path)
            #sdd_confidence_drop_avg.append(drop_case1)
            #random_confidence_drop_avg.append(drop_case2)
            original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence = ensemble_data(image_path) 
            original_top_class_all.append(original_top_class)
            original_top_prob_all.append(original_top_prob)
            perturbed_top_class_all.append(perturbed_top_class)
            perturbed_top_prob_all.append(perturbed_top_prob)
            perturbed_confidence_all.append(perturbed_confidence)
            random_perturbed_top_class_all.append(random_perturbed_top_class)
            random_perturbed_top_prob_all.append(random_perturbed_top_prob)
            random_perturbed_confidence_all.append(random_perturbed_confidence)
            
            
            
  

    # Combine lists into a DataFrame
    df = pd.DataFrame({
           'Image': image_name,
           'Original_Prediction': original_top_class_all,
           'Original_Prob': original_top_prob_all,
           'Ensemble_Pertb_Prediction': perturbed_top_class_all,
           'Ensemble_Pertb_Prob': perturbed_top_prob_all,
           'Ensemble_Pertb_Prob_Original_Pred': perturbed_confidence_all,
           'Random_Pertb_Prediction': random_perturbed_top_class_all,
           'Random_Pertb_Prob': random_perturbed_top_prob_all,
           'Random_Pertb_Prob_Original_Pred': random_perturbed_confidence_all
                         })

    print(df)
    
    return df


df = process_folder_two(root_folder)
        




In [ ]:
def process_folder_mask_two(folder_path):
    """
    Process all images in a folder and its subfolders, calculate average confidence drops.
    """
    #sdd_confidence_drop_avg = []
    #random_confidence_drop_avg = []
    image_name = []
    original_top_class_all = []
    original_top_prob_all = []
    perturbed_top_class_all = []
    perturbed_top_prob_all = []
    perturbed_confidence_all = []
    random_perturbed_top_class_all = []
    random_perturbed_top_prob_all = []
    random_perturbed_confidence_all = []

    for subdir, _, files in os.walk(folder_path):
       for file in files:  # Loop through each file in the current directory
            image_name.append(file)
            image_path = os.path.join(subdir, file)
            #drop_case1, drop_case2 = sdd_generate(image_path)
            #sdd_confidence_drop_avg.append(drop_case1)
            #random_confidence_drop_avg.append(drop_case2)
            original_top_class, original_top_prob, perturbed_top_class, perturbed_top_prob, perturbed_confidence, random_perturbed_top_class, random_perturbed_top_prob, random_perturbed_confidence = ensemble_mask_data(image_path) 
            original_top_class_all.append(original_top_class)
            original_top_prob_all.append(original_top_prob)
            perturbed_top_class_all.append(perturbed_top_class)
            perturbed_top_prob_all.append(perturbed_top_prob)
            perturbed_confidence_all.append(perturbed_confidence)
            random_perturbed_top_class_all.append(random_perturbed_top_class)
            random_perturbed_top_prob_all.append(random_perturbed_top_prob)
            random_perturbed_confidence_all.append(random_perturbed_confidence)
            
            
            
  

    # Combine lists into a DataFrame
    df = pd.DataFrame({
           'Image': image_name,
           'Original_Prediction': original_top_class_all,
           'Original_Prob': original_top_prob_all,
           'Ensemble_Pertb_Prediction': perturbed_top_class_all,
           'Ensemble_Pertb_Prob': perturbed_top_prob_all,
           'Ensemble_Pertb_Prob_Original_Pred': perturbed_confidence_all,
           'Random_Pertb_Prediction': random_perturbed_top_class_all,
           'Random_Pertb_Prob': random_perturbed_top_prob_all,
           'Random_Pertb_Prob_Original_Pred': random_perturbed_confidence_all
                         })

    print(df)
    
    return df


df_2 = process_folder_mask_two(root_folder)
        




In [ ]:
# Count the number of differences
num_differences_1 = (df_2['Original_Prediction'] != df_2['Ensemble_Pertb_Prediction']).sum()

# Calculate the percentage of differences
percentage_differences_1 = (num_differences_1 / len(df_2['Original_Prediction'])) * 100

print(f"Percentage of differences for Ensemble: {percentage_differences_1:.2f}%")


# Count the number of differences
num_differences_2 = (df_2['Original_Prediction'] != df_2['Random_Pertb_Prediction']).sum()

# Calculate the percentage of differences
percentage_differences_2 = (num_differences_2 / len(df_2['Original_Prediction'])) * 100

print(f"Percentage of differences for Random: {percentage_differences_2:.2f}%")


In [ ]:
# Subtract Column2 from Column1
difference_1 = df_2['Original_Prob'] - df_2['Ensemble_Pertb_Prob_Original_Pred']
difference_2 = df_2['Original_Prob'] - df_2['Random_Pertb_Prob_Original_Pred']

# Calculate the average of the difference
average_difference_1 = difference_1.mean()
average_difference_2 = difference_2.mean()

print("Ensemble Confidence Drop:", average_difference_1)
print("Random Confidence Drop:", average_difference_2)

In [ ]:
import scipy.stats as stats

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# List of column names and titles
columns = ['Original_Prob', 'Ensemble_Pertb_Prob_Original_Pred'] #'Random_Pertb_Prob_Original_Pred']
titles = ['Original Probability', 'Ensemble_CAM Retention Prob Original Pred'] #'Random Retention Prob Original Pred']

for i, col in enumerate(columns):
    # Compute mean and standard deviation
    mean, std = df_2[col].mean(), df_2[col].std()

    # Generate x values for the bell curve
    x = np.linspace(df_2[col].min(), df_2[col].max(), 100)
    y = stats.norm.pdf(x, mean, std)

    # Plot bell curve
    axes[i].plot(x, y, 'r-', lw=2)
    

    # Set labels
    axes[i].set_title(titles[i])
    axes[i].set_xlabel('Probability')
    axes[i].set_ylabel('Density')
    
    # Ensure y-axis scale goes up to 6
    axes[i].set_ylim(0, 6)  # Keeps the upper limit at 6
    axes[i].set_yticks(np.arange(0, 6.2, 0.4))  # Ticks from 0 to 6 with a step of 0.2

plt.tight_layout()
plt.show()